In [1]:
import os
import csv
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd
import cv2 

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!tar -xf "/content/fer2013.tar.gz"

In [4]:
from keras.utils import np_utils
data = pd.read_csv('/content/fer2013/fer2013.csv')
train = data[data.Usage == "Training"]
test = data[data.Usage == "PrivateTest"]
test_pub = data[data.Usage == "PublicTest"]


train_labels = train["emotion"].values
test_labels = test["emotion"].values
test_pub_labels = test_pub["emotion"].values

train_ds = train["pixels"].values
test_ds = test["pixels"].values
test_pub_ds = test_pub["pixels"].values

train_ds_list = list()
test_ds_list = list()
test_pub_ds_list = list()

for i in range(28709):
  train_ds_list.append(np.array(train_ds[i].split(" ")))
  train_ds_list[i] = train_ds_list[i].astype(np.float)

for i in range(3589):
  test_ds_list.append(np.array(test_ds[i].split(" ")))
  test_ds_list[i] = test_ds_list[i].astype(np.float)

for i in range(3589):
  test_pub_ds_list.append(np.array(test_pub_ds[i].split(" ")))
  test_pub_ds_list[i] = test_pub_ds_list[i].astype(np.float)

train_np = np.array(train_ds_list)
test_np = np.array(test_ds_list)
test_pub_np = np.array(test_pub_ds_list)



train_np = train_np.reshape(train_np.shape[0], 48, 48, 1)
test_np = test_np.reshape(test_np.shape[0],48, 48, 1)
test_pub_np = test_pub_np.reshape(test_pub_np.shape[0], 48, 48, 1)

train_np = np.repeat(train_np , 3, axis=3)
test_np = np.repeat(test_np , 3, axis=3)
test_pub_np = np.repeat(test_pub_np , 3, axis=3)
print(train_np.shape)

train_labels = np_utils.to_categorical(train_labels, 7)
test_labels = np_utils.to_categorical(test_labels, 7)
test_pub_labels = np_utils.to_categorical(test_pub_labels, 7) 


(28709, 48, 48, 3)


In [5]:
base_model = tf.keras.applications.resnet_v2.ResNet50V2(weights='imagenet',include_top=False)
base_model.trainable = True
#base_model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
x = base_model.output
 
# Adding a Global Average Pooling layer
x = layers.GlobalAveragePooling2D()(x)
 
# Adding a fully connected layer having 1024 neurons
x = layers.Dense(2048, activation='relu')(x)
# Adding a fully connected layer having 2 neurons which will
# give the probability of image having either dog or cat
predictions = layers.Dense(7, activation='softmax')(x)
 
# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, None, None, 6 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [0]:
from tensorflow.keras.optimizers import RMSprop,SGD
base_learning_rate=.0001
model.compile(tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics = ['accuracy'])
#model.compile(optimizer = RMSprop(lr=0.0001), 
#              loss = 'categorical_crossentropy', 
#              metrics = ['acc'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator
# parameters
num_epochs = 70
verbose = 1
batch_size = 64


train_big = np.vstack([train_np, test_pub_np])
train_label_big = np.concatenate([train_labels, test_pub_labels])
# data generator
data_generator = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [10]:
model.fit_generator(data_generator.flow(train_big, train_label_big),
                        steps_per_epoch=len(train_big) / batch_size,
                        epochs=num_epochs, verbose=1, 
                    validation_data=data_generator.flow(test_np, test_labels)
                        )

Epoch 1/70
505/504 [==============================] - 46s 90ms/step - loss: 1.7577 - acc: 0.2940 - val_loss: 1.6657 - val_acc: 0.3408
Epoch 2/70
505/504 [==============================] - 33s 65ms/step - loss: 1.5758 - acc: 0.3870 - val_loss: 1.5199 - val_acc: 0.4132
Epoch 3/70
505/504 [==============================] - 32s 64ms/step - loss: 1.4806 - acc: 0.4270 - val_loss: 1.4663 - val_acc: 0.4500
Epoch 4/70
505/504 [==============================] - 32s 64ms/step - loss: 1.4138 - acc: 0.4627 - val_loss: 1.3841 - val_acc: 0.4670
Epoch 5/70
505/504 [==============================] - 32s 64ms/step - loss: 1.3674 - acc: 0.4782 - val_loss: 1.3827 - val_acc: 0.4692
Epoch 6/70
505/504 [==============================] - 33s 65ms/step - loss: 1.3443 - acc: 0.4860 - val_loss: 1.3208 - val_acc: 0.4985
Epoch 7/70
505/504 [==============================] - 32s 64ms/step - loss: 1.3166 - acc: 0.4951 - val_loss: 1.3140 - val_acc: 0.4968
Epoch 8/70
505/504 [==============================] - 33s 64ms

In [0]:
model.save_weights("weights_70_epochs_Resnet50.hdf5")
